# import & read

In [1]:
import pandas as pd
from datetime import time
from datetime import timedelta

In [2]:
data=pd.read_csv('flights.csv')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2140\1255340642.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('flights.csv')


In [3]:
ap=pd.read_csv('AIRPORTID.csv', encoding='gbk')

In [4]:
null_counts = data.isnull().sum()
print(null_counts.to_dict())

{'YEAR': 0, 'MONTH': 0, 'DAY': 0, 'DAY_OF_WEEK': 0, 'AIRLINE': 0, 'FLIGHT_NUMBER': 0, 'TAIL_NUMBER': 14721, 'ORIGIN_AIRPORT': 0, 'DESTINATION_AIRPORT': 0, 'SCHEDULED_DEPARTURE': 0, 'DEPARTURE_TIME': 86153, 'DEPARTURE_DELAY': 86153, 'TAXI_OUT': 89047, 'WHEELS_OFF': 89047, 'SCHEDULED_TIME': 6, 'ELAPSED_TIME': 105071, 'AIR_TIME': 105071, 'DISTANCE': 0, 'WHEELS_ON': 92513, 'TAXI_IN': 92513, 'SCHEDULED_ARRIVAL': 0, 'ARRIVAL_TIME': 92513, 'ARRIVAL_DELAY': 105071, 'DIVERTED': 0, 'CANCELLED': 0, 'CANCELLATION_REASON': 5729195, 'AIR_SYSTEM_DELAY': 4755640, 'SECURITY_DELAY': 4755640, 'AIRLINE_DELAY': 4755640, 'LATE_AIRCRAFT_DELAY': 4755640, 'WEATHER_DELAY': 4755640}


# "code" processing

In [5]:
lookup_dict = ap.set_index('Code')['IATA_CODE'].to_dict()


In [6]:
data['ORIGIN_AIRPORT_IATA'] = data['ORIGIN_AIRPORT'].apply(pd.to_numeric, errors='coerce').map(lookup_dict)
data['ORIGIN_AIRPORT_IATA'].fillna(data['ORIGIN_AIRPORT'],inplace=True)

In [7]:
data['DESTINATION_AIRPORT_IATA'] = data['DESTINATION_AIRPORT'].apply(pd.to_numeric, errors='coerce').map(lookup_dict)
data['DESTINATION_AIRPORT_IATA'].fillna(data['DESTINATION_AIRPORT'],inplace=True)

# datetime processing

In [8]:
data['Date'] = pd.to_datetime(data['YEAR'].astype(str) + '-' + data['MONTH'].astype(str) + '-' + data['DAY'].astype(str))

In [9]:
timelist=['SCHEDULED_DEPARTURE','DEPARTURE_TIME','WHEELS_OFF','WHEELS_ON','SCHEDULED_ARRIVAL','ARRIVAL_TIME']
tl=['SD','DT','WF','WN','SA','AT']

def convert_to_time(time_int):
    hours = time_int // 100
    minutes = time_int % 100
    return time(hours, minutes)

for m in range(6):
    i=timelist[m]
    j=tl[m]
    data[i]= data[i].replace(2400, 0)
    data.loc[data[i].notna(),j]= data.loc[data[i].notna(),i].astype(int).apply(convert_to_time)
    #data.loc[data[j].notna(),j]=pd.to_datetime(data.loc[data[j].notna(),'Date'].astype(str)+' '+data.loc[data[j].notna(),j].astype(str))


In [10]:
data.loc[data['SD'].notna(),'SD']=pd.to_datetime(data.loc[data['SD'].notna(),'Date'].astype(str)\
                                                 +' '+data.loc[data['SD'].notna(),'SD'].astype(str))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2140\2962505970.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[data['SD'].notna(),'SD']=pd.to_datetime(data.loc[data['SD'].notna(),'Date'].astype(str)\


In [11]:
mask1=(data['DEPARTURE_DELAY']>(1439-data['SCHEDULED_DEPARTURE']//100*60-data['SCHEDULED_DEPARTURE']%100)) &(data['DEPARTURE_TIME'].notna())
mask2=(data['DEPARTURE_DELAY']<=(1439-data['SCHEDULED_DEPARTURE']//100*60-data['SCHEDULED_DEPARTURE']%100))\
    & (data['DEPARTURE_DELAY']+data['SCHEDULED_DEPARTURE']//100*60+data['SCHEDULED_DEPARTURE']%100>=0)\
        &(data['DEPARTURE_TIME'].notna())
mask3=(data['DEPARTURE_DELAY']+data['SCHEDULED_DEPARTURE']//100*60+data['SCHEDULED_DEPARTURE']%100<0)&(data['DEPARTURE_TIME'].notna())

In [12]:
data.loc[mask1,'DT']=pd.to_datetime(data.loc[mask1,'Date'].astype(str)+' '+data.loc[mask1,'DT'].astype(str))+timedelta(days=1)
data.loc[mask2,'DT']=pd.to_datetime(data.loc[mask2,'Date'].astype(str)+' '+data.loc[mask2,'DT'].astype(str))
data.loc[mask3,'DT']=pd.to_datetime(data.loc[mask3,'Date'].astype(str)+' '+data.loc[mask3,'DT'].astype(str))+timedelta(days=-1)

In [13]:
mask4=(mask1 | (mask2 & (data['DEPARTURE_TIME']>data['WHEELS_OFF']))) & (data['WHEELS_OFF'].notna())
mask5=((mask2 & (data['DEPARTURE_TIME']<=data['WHEELS_OFF']))|(mask3 & (data['DEPARTURE_TIME']>data['WHEELS_OFF'])))& (data['WHEELS_OFF'].notna())
mask6=(mask3 & (data['DEPARTURE_TIME']<=data['WHEELS_OFF']))& (data['WHEELS_OFF'].notna())

In [14]:
data.loc[mask4,'WF']=pd.to_datetime(data.loc[mask4,'Date'].astype(str)+' '+data.loc[mask4,'WF'].astype(str))+timedelta(days=1)
data.loc[mask5,'WF']=pd.to_datetime(data.loc[mask5,'Date'].astype(str)+' '+data.loc[mask5,'WF'].astype(str))
data.loc[mask6,'WF']=pd.to_datetime(data.loc[mask6,'Date'].astype(str)+' '+data.loc[mask6,'WF'].astype(str))+timedelta(days=-1)

In [15]:
mask7=(mask1 | (mask2 & (data['DEPARTURE_TIME']>data['WHEELS_ON'])&\
                (data['DEPARTURE_TIME']-data['WHEELS_ON']+data['TAXI_OUT']+data['AIR_TIME']>60))) & (data['WHEELS_ON'].notna())
mask8=(~(mask1 | (mask2 & (data['DEPARTURE_TIME']>data['WHEELS_ON'])&\
                (data['DEPARTURE_TIME']-data['WHEELS_ON']+data['TAXI_OUT']+data['AIR_TIME']>60))))& (data['WHEELS_ON'].notna())

In [16]:
data.loc[mask7,'WN']=pd.to_datetime(data.loc[mask7,'Date'].astype(str)+' '+data.loc[mask7,'WN'].astype(str))+timedelta(days=1)
data.loc[mask8,'WN']=pd.to_datetime(data.loc[mask8,'Date'].astype(str)+' '+data.loc[mask8,'WN'].astype(str))

In [17]:
mask9=((data['SCHEDULED_DEPARTURE']>data['SCHEDULED_ARRIVAL'])&(data['SCHEDULED_DEPARTURE']-data['SCHEDULED_ARRIVAL']>60-data['SCHEDULED_TIME']))|\
    ((data['SCHEDULED_TIME'].isnull())&(data['SCHEDULED_DEPARTURE']>data['SCHEDULED_ARRIVAL']))
mask10=~mask9

In [18]:
data.loc[mask9,'SA']=pd.to_datetime(data.loc[mask9,'Date'].astype(str)+' '+data.loc[mask9,'SA'].astype(str))+timedelta(days=1)
data.loc[mask10,'SA']=pd.to_datetime(data.loc[mask10,'Date'].astype(str)+' '+data.loc[mask10,'SA'].astype(str))

In [19]:
mask11=(mask1 | (mask2 & (data['DEPARTURE_TIME']>data['ARRIVAL_TIME'])&\
                 (data['DEPARTURE_TIME']-data['ARRIVAL_TIME']+data['ELAPSED_TIME']>60))) & (data['ARRIVAL_TIME'].notna())
mask12=(~(mask1 | (mask2 & (data['DEPARTURE_TIME']>data['ARRIVAL_TIME'])&\
                 (data['DEPARTURE_TIME']-data['ARRIVAL_TIME']+data['ELAPSED_TIME']>60))))& (data['ARRIVAL_TIME'].notna())

In [20]:
data.loc[mask11,'AT']=pd.to_datetime(data.loc[mask11,'Date'].astype(str)+' '+data.loc[mask11,'AT'].astype(str))+timedelta(days=1)
data.loc[mask12,'AT']=pd.to_datetime(data.loc[mask12,'Date'].astype(str)+' '+data.loc[mask12,'AT'].astype(str))

In [21]:
#test
for t in tl:
    l=[]
    for i in data[t]:
        if type(i) not in l:
            l.append(type(i))
            print(i,type(i))


2015-01-01 00:05:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2014-12-31 23:54:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
nan <class 'float'>
2015-01-01 00:15:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
nan <class 'float'>
2015-01-01 04:04:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
nan <class 'float'>
2015-01-01 04:30:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2015-01-01 04:08:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
nan <class 'float'>


# reason processing

In [22]:
delayreason=['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']

In [23]:
maskr=((data['ARRIVAL_DELAY']>0) & (data['AIR_SYSTEM_DELAY'].isnull()))| (data['ARRIVAL_DELAY']<=0)
#not included: diverted/cancelled

In [24]:
for i in delayreason:
    data.loc[maskr,i]=0

# 输出

In [25]:
data.describe()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,5819079.0,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.732926e+06,5.732926e+06,5.730032e+06,5.730032e+06,...,5.819079e+06,5.726566e+06,5.714008e+06,5.819079e+06,5.819079e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06,5.714008e+06
mean,2015.0,6.524085e+00,1.570459e+01,3.926941e+00,2.173093e+03,1.329602e+03,1.334990e+03,9.370158e+00,1.607166e+01,1.356866e+03,...,1.493807e+03,1.475462e+03,4.407057e+00,2.609863e-03,1.544643e-02,2.508880e+00,1.417306e-02,3.530439e+00,4.368550e+00,5.425671e-01
std,0.0,3.405137e+00,8.783425e+00,1.988845e+00,1.757064e+03,4.837518e+02,4.964817e+02,3.708094e+01,8.895574e+00,4.981054e+02,...,5.071652e+02,5.268592e+02,3.927130e+01,5.102012e-02,1.233201e-01,1.317102e+01,9.251752e-01,2.204992e+01,2.075421e+01,8.887775e+00
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-8.200000e+01,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,-8.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2015.0,4.000000e+00,8.000000e+00,2.000000e+00,7.300000e+02,9.170000e+02,9.210000e+02,-5.000000e+00,1.100000e+01,9.350000e+02,...,1.110000e+03,1.058000e+03,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2015.0,7.000000e+00,1.600000e+01,4.000000e+00,1.690000e+03,1.325000e+03,1.330000e+03,-2.000000e+00,1.400000e+01,1.343000e+03,...,1.520000e+03,1.512000e+03,-5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2015.0,9.000000e+00,2.300000e+01,6.000000e+00,3.230000e+03,1.730000e+03,1.740000e+03,7.000000e+00,1.900000e+01,1.754000e+03,...,1.918000e+03,1.916000e+03,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.855000e+03,2.359000e+03,2.359000e+03,1.988000e+03,2.250000e+02,2.359000e+03,...,2.359000e+03,2.359000e+03,1.971000e+03,1.000000e+00,1.000000e+00,1.134000e+03,5.730000e+02,1.971000e+03,1.331000e+03,1.211000e+03


In [26]:
data.to_csv('process.csv')